In [1]:
import cv2
import numpy as np
import pandas as pd
table_image = cv2.imread(r"C:\Users\weili\Pictures\Screenshots\Screenshot 2023-12-18 154231.png")

def preprocess(img, factor: int):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = Image.fromarray(img)
    enhancer = ImageEnhance.Sharpness(img).enhance(factor)
    if gray.std() < 30:
        enhancer = ImageEnhance.Contrast(enhancer).enhance(factor)
    return np.array(enhancer)

gray = cv2.cvtColor(table_image, cv2.COLOR_BGR2GRAY)
thresh, img_bin = cv2.threshold(
    gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
img_bin = 255-img_bin

kernel_len = gray.shape[1]//120
hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (kernel_len, 1))
image_horizontal = cv2.erode(img_bin, hor_kernel, iterations=3)
horizontal_lines = cv2.dilate(image_horizontal, hor_kernel, iterations=3)

h_lines = cv2.HoughLinesP(
    horizontal_lines, 1, np.pi/180, 30, maxLineGap=250)



In [2]:
def group_h_lines(h_lines, thin_thresh):
    new_h_lines = []
    while len(h_lines) > 0:
        thresh = sorted(h_lines, key=lambda x: x[0][1])[0][0]
        lines = [line for line in h_lines if thresh[1] -
                 thin_thresh <= line[0][1] <= thresh[1] + thin_thresh]
        h_lines = [line for line in h_lines if thresh[1] - thin_thresh >
                   line[0][1] or line[0][1] > thresh[1] + thin_thresh]
        x = []
        for line in lines:
            x.append(line[0][0])
            x.append(line[0][2])
        x_min, x_max = min(x) - int(5*thin_thresh), max(x) + int(5*thin_thresh)
        new_h_lines.append([x_min, thresh[1], x_max, thresh[1]])
    return new_h_lines
    
new_horizontal_lines = group_h_lines(h_lines, kernel_len)


In [3]:
kernel_len = gray.shape[1]//120
ver_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, kernel_len))
image_vertical = cv2.erode(img_bin, ver_kernel, iterations=3)
vertical_lines = cv2.dilate(image_vertical, ver_kernel, iterations=3)

v_lines = cv2.HoughLinesP(vertical_lines, 1, np.pi/180, 30, maxLineGap=250)

def group_v_lines(v_lines, thin_thresh):
    new_v_lines = []
    while len(v_lines) > 0:
        thresh = sorted(v_lines, key=lambda x: x[0][0])[0][0]
        lines = [line for line in v_lines if thresh[0] -
                 thin_thresh <= line[0][0] <= thresh[0] + thin_thresh]
        v_lines = [line for line in v_lines if thresh[0] - thin_thresh >
                   line[0][0] or line[0][0] > thresh[0] + thin_thresh]
        y = []
        for line in lines:
            y.append(line[0][1])
            y.append(line[0][3])
        y_min, y_max = min(y) - int(4*thin_thresh), max(y) + int(4*thin_thresh)
        new_v_lines.append([thresh[0], y_min, thresh[0], y_max])
    return new_v_lines
    
new_vertical_lines = group_v_lines(v_lines, kernel_len)


In [4]:
def seg_intersect(line1: list, line2: list):
    a1, a2 = line1
    b1, b2 = line2
    da = a2-a1
    db = b2-b1
    dp = a1-b1

    def perp(a):
        b = np.empty_like(a)
        b[0] = -a[1]
        b[1] = a[0]
        return b

    dap = perp(da)
    denom = np.dot(dap, db)
    num = np.dot(dap, dp)
    return (num / denom.astype(float))*db + b1


In [5]:
points = []
for hline in new_horizontal_lines:
    x1A, y1A, x2A, y2A = hline
    for vline in new_vertical_lines:
        x1B, y1B, x2B, y2B = vline

        line1 = [np.array([x1A, y1A]), np.array([x2A, y2A])]
        line2 = [np.array([x1B, y1B]), np.array([x2B, y2B])]

        x, y = seg_intersect(line1, line2)
        if x1A <= x <= x2A and y1B <= y <= y2B:
            points.append([int(x), int(y)])


In [6]:
def get_bottom_right(right_points, bottom_points, points):
    for right in right_points:
        for bottom in bottom_points:
            if [right[0], bottom[1]] in points:
                return right[0], bottom[1]
    return None, None

cells = []
for point in points:
    left, top = point
    right_points = sorted(
        [p for p in points if p[0] > left and p[1] == top], key=lambda x: x[0])
    bottom_points = sorted(
        [p for p in points if p[1] > top and p[0] == left], key=lambda x: x[1])

    right, bottom = get_bottom_right(
        right_points, bottom_points, points)
    if right and bottom:
        cv2.rectangle(table_image, (left, top), (right, bottom), (0, 0, 255), 2)
        cells.append([left, top, right, bottom])


In [12]:
!pip uninstall opencv-python-headless -y
!pip install opencv-python --upgrade

   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 1.1 MB/s eta 0:00:35
   ---------------------------------------- 0.3/38.1 MB 3.5 MB/s eta 0:00:11
    --------------------------------------- 0.8/38.1 MB 5.8 MB/s eta 0:00:07
   - -------------------------------------- 1.1/38.1 MB 6.3 MB/s eta 0:00:06
   - -------------------------------------- 1.8/38.1 MB 7.4 MB/s eta 0:00:05
   -- ------------------------------------- 2.5/38.1 MB 8.7 MB/s eta 0:00:05
   --- ------------------------------------ 3.0/38.1 MB 9.0 MB/s eta 0:00:04
   --- ------------------------------------ 3.6/38.1 MB 9.4 MB/s eta 0:00:04
   ---- ----------------------------------- 4.0/38.1 MB 9.5 MB/s eta 0:00:04
   ---- ----------------------------------- 4.6/38.1 MB 9.8 MB/s eta 0:00:04
   ----- ---------------------------------- 5.0/38.1 MB 9.8 MB/s eta 0:00:04
   ----- ---------------------------------- 5.5/38.1 MB 9.7 MB/s eta 0:00:04
   ---

In [13]:
cv2.imshow('anh 1 day', table_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.6.0) C:\b\abs_f8n1j3l9l0\croot\opencv-suite_1691622637237\work\modules\highgui\src\window.cpp:1267: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


In [ ]:
import pytesseract
import cv2
import pandas as pd

# Assuming you have already defined the 'cells' list and 'table_image' variable

tt = []
data = {'Text': []}

for cell in cells:
    left, top, right, bottom = cell
    roi = table_image[top:bottom, left:right]

    # Chuyển ảnh sang đen trắng để cải thiện khả năng nhận diện
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Áp dụng ngưỡng để tách nền và chữ
    _, threshold_roi = cv2.threshold(gray_roi, 150, 255, cv2.THRESH_BINARY)

    # Sử dụng Tesseract để nhận diện văn bản
    text = pytesseract.image_to_string(threshold_roi, lang='vie')

    # In kết quả
    print("Text in bounding box:", text)
    tt.append(text)

    # Vẽ bounding box lên ảnh gốc
    cv2.rectangle(table_image, (left, top), (right, bottom), (0, 0, 255), 2)

    # Append text to the DataFrame
    data['Text'].append(text)

# Create a DataFrame
df = pd.DataFrame(data)

# Now 'df' contains the recognized text
print(df)


In [ ]:

for cell in cells:
    left, top, right, bottom = cell
    roi = table_image[top:bottom, left:right]

    # Chuyển ảnh sang đen trắng để cải thiện khả năng nhận diện
    gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # Áp dụng ngưỡng để tách nền và chữ
    _, threshold_roi = cv2.threshold(gray_roi, 150, 255, cv2.THRESH_BINARY)

    # Sử dụng Tesseract để nhận diện văn bản
    text = pytesseract.image_to_string(threshold_roi, lang='vie')

    # In kết quả
    print("Text in bounding box:", text)

    # Vẽ bounding box lên ảnh gốc
    cv2.rectangle(table_image, (left, top), (right, bottom), (0, 0, 255), 2)

# # Hiển thị ảnh kết quả (để kiểm tra)
# cv2.imshow("Result Image", table_image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
text